# XSL Script Processing
Extensible Stylesheet Language (XSL) files are commonly used to describe the processing and rendering of data within XML files, rule detects when adversaries abuse this functionality to execute arbitrary files while potentially bypassing application whitelisting defenses

## Rule Content
```
- title: XSL Script Processing
  id: 05c36dd6-79d6-4a9a-97da-3db20298ab2d
  status: experimental
  description: Extensible Stylesheet Language (XSL) files are commonly used to describe
    the processing and rendering of data within XML files, rule detects when adversaries
    abuse this functionality to execute arbitrary files while potentially bypassing
    application whitelisting defenses
  author: Timur Zinniatullin, oscd.community
  date: 2019/10/21
  modified: 2019/11/04
  references:
  - https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1220/T1220.yaml
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
    - Image|endswith: \wmic.exe
      CommandLine|contains: /format
    - Image|endswith: \msxsl.exe
    condition: selection
  falsepositives:
  - WMIC.exe FP depend on scripts and administrative methods used in the monitored
    environment
  - msxsl.exe is not installed by default so unlikely.
  level: medium
  tags:
  - attack.execution
  - attack.t1220

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:*\\wmic.exe AND process_command_line.keyword:*\/format*) OR process_path.keyword:*\\msxsl.exe)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()